In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp

plt.rcParams['figure.figsize'] = (16,16)

def mosthighlycorrelated(mydataframe, numtoreport):
    # find the correlations
    cormatrix = mydataframe.corr()
    # set the correlations on the diagonal or lower triangle to zero,
    # so they will not be reported as the highest ones:
    cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
    # find the top n correlations
    cormatrix = cormatrix.stack()
    cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
    # assign human-friendly names
    cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]
    return cormatrix.head(numtoreport)

In [2]:
df1=pd.read_csv("DataNN.txt")
df2 = df1.copy()
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-1].copy()
#df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))+0.01 #Boxcox values must be positive
df1 = (df1-np.mean(df1, axis = 0))/(np.std(df1, axis = 0)) #Boxcox values must be positive

df1

,Mean_FCC1,Mean_FCC2,Mean_FCC3,Mean_FCC4,Mean_FCC5,Mean_FCC6,Mean_FCC7,Mean_FCC8,Mean_FCC9,Mean_FCC10,...,DER2_FCC7,DER2_FCC8,DER2_FCC9,DER2_FCC10,DER2_FCC11,DER2_FCC12,LENGTH,MINFREC,MAXFREC,DOMFREC
0,-0.037072,0.151833,0.467133,-0.243469,0.377079,-0.330127,0.359430,-0.206106,0.290794,-0.100906,...,-0.224014,0.008211,-0.144813,0.012702,-1.073404,-0.012702,-0.442405,0.147298,-0.206577,-0.029646
1,-0.674306,-0.702380,0.930034,0.003890,0.884340,-0.365400,0.904189,-0.367941,0.738815,-0.212006,...,-0.064666,-0.023763,-0.066813,-0.001497,-0.399401,0.001497,-0.721771,0.086257,-0.287262,-0.030568
2,0.796588,-0.051883,1.175866,-1.259668,0.269421,-0.863876,-0.593208,-0.171436,0.315916,-0.367398,...,-0.953643,0.722114,0.885455,0.116261,-0.763994,-0.116261,0.535319,0.103726,0.107129,0.356606
3,0.080494,-0.009501,0.556963,-0.310354,0.300041,-0.455534,0.435348,-0.442042,0.405029,-0.234277,...,-0.063736,-0.182047,-0.054460,-0.056998,-0.087690,0.056998,-0.651924,0.121145,-0.197647,-0.047787
4,0.617746,0.174778,-0.054875,-1.160769,1.566239,0.987477,2.086930,0.827960,1.453625,-0.042288,...,-1.903178,-0.799436,-0.182161,-0.528774,-0.682836,0.528774,0.605166,-0.000887,-0.009381,-0.016475
5,0.371101,0.372037,0.482308,-0.994398,-0.016035,0.130452,0.678027,-0.817117,0.297945,-0.297373,...,-0.365472,0.073549,0.057508,-0.389711,-0.691257,0.389711,-0.163062,0.112410,-0.009381,0.031698
6,0.551394,-0.125115,1.194746,-1.377535,-0.194122,-2.188093,0.146236,-0.331491,0.107014,-1.215254,...,0.219667,1.935301,-0.132219,0.069375,-0.891542,-0.069375,-0.163062,0.112410,0.080234,0.332315
7,-1.317921,-0.378291,0.713237,-0.232893,0.645394,-0.449011,0.707309,-0.414363,0.596022,-0.229405,...,-0.252132,0.032368,-0.164855,0.022637,-1.910211,-0.022637,-1.280462,0.103726,-0.305175,-0.057114
8,0.832756,-0.017534,1.491100,-1.074713,-0.673646,-1.339109,-2.225026,-0.554582,1.684673,-0.271037,...,-1.064880,-1.430137,0.469709,-0.531607,-0.573725,0.531607,0.046457,0.103726,0.178832,0.022883
9,-0.256977,-1.290500,0.284035,0.040192,0.972080,-0.237964,0.910769,-0.177331,0.875587,-0.018624,...,0.555129,0.354915,-0.037756,-0.052973,-0.740431,0.052973,-0.931267,0.060104,-0.224490,0.020269


In [3]:
NumVariables = df1.shape[1]
NumVariables

76

In [4]:
X = np.array(df1)
y = np.array(classes)-1.0
X

array([[-0.03707245,  0.15183306,  0.46713345, ...,  0.14729848,
        -0.206577  , -0.02964553],
       [-0.6743058 , -0.70238042,  0.93003352, ...,  0.08625698,
        -0.28726215, -0.03056798],
       [ 0.79658832, -0.0518832 ,  1.1758662 , ...,  0.10372643,
         0.10712897,  0.35660579],
       ...,
       [-0.66714486, -0.58095343,  0.31484866, ..., -1.64101437,
        -2.09197758, -1.83155378],
       [-0.68563641, -0.81526349, -0.09270497, ..., -1.65783379,
        -2.08429029, -1.83197401],
       [-1.02492012, -0.68550854,  0.32213635, ..., -1.64194878,
        -2.08429029, -1.8297345 ]])

In [5]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def d_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def RedNeuronalTrain(X,y,Lsizes,alpha = .1,epochs = 20,outputs = 2):
    N  = X.shape[0]   #número de muestras para realizar el entrenamiento
    nf = X.shape[1]   #número de características
    L = len(Lsizes)   #número de capas ocultas
    w_ = []           #pesos de la red neuronal
    
    w_.append(np.random.rand(Lsizes[0],nf+1)) #Primera capa: L_1 filas & nf + 1 columnas (añadiendo la neurona de BIAS)
    for k in range(0,L-1):
        w_.append(np.random.rand(Lsizes[k+1],(Lsizes[k]+1)))#capa siguiente en las filas y capa actual columnas
    w_.append(np.random.rand(outputs,Lsizes[L-1]+1))        #Ultima capa interna: ourput_N flas & L_final columnas 
                                                            #(añadiendo la neurona de BIAS) 
    for j in range(0,epochs): #condición de parada
        J = 0                 #error cuadratico medio  
        for i in range(0,N):
            #One hot encoding
            Y = np.zeros(shape=(outputs,1))
            Y[int(y[i])] = 1.0
            a_ = []
            z_ = []
            #Forward
            #Para cada muestra
            a_.append(X[i,None].T)
            #Se agrega el bias de la capa entrada
            for k in range(0,(L+1)):
                #Se agrega el bias a la capa externa k+1
                a_[k] = np.insert(a_[k], 0, 1,axis=0)
                z_.append(np.dot(w_[k],a_[k]))
                a_.append(sigmoid(z_[k]))
            #Backward
            J = J + np.sum((Y-a_[len(a_)-1])*(Y-a_[len(a_)-1]))
            d_=clearList(len(a_))
            
            d_[L+1]= (Y-a_[L+1])*d_sigmoid(z_[L])
            for k in range(L,0,-1):
                d_[k] = np.dot(w_[k].T,d_[k+1])
                d_[k] = np.delete(d_[k],0,axis=0)
                d_[k] = d_[k]*d_sigmoid(z_[k-1])
                
            DE_dw_=clearList(len(w_))
            for k in range(0,len(w_)):
                DE_dw_[k] = -np.dot(d_[k+1],a_[k].T)
            for k in range(0,len(w_)):
                w_[k] = w_[k]-alpha*DE_dw_[k]
            #print(np.array(Y) == np.array(a_[len(a_)-1]))
        #print(a_[len(a_)-1])
    return [w_,J/2]

def Predict(X,w_): 
    #Forward
    #Para cada muestra
    L = len(w_)
    a_ = []
    z_ = []
    a_.append(X[:,None])
    for k in range(0,(L)):
        #Se agrega el bias a la capa externa k+1
        a_[k] = np.insert(a_[k], 0, 1,axis=0)
        z_.append(np.dot(w_[k],a_[k]))
        a_.append(sigmoid(z_[k]))
    return a_[len(a_)-1]


In [6]:
index=np.random.permutation(X.shape[0])
X1    = X[index,:]
y1    = y[index  ]

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_train = X1[0:Ntrain,:]
X_val   = X1[Ntrain:(Ntrain+Nval),:]
X_test  = X1[(Nsamples-Ntest):Nsamples,:]
y_train = y1[0:Ntrain  ]
y_val   = y1[Ntrain:(Ntrain+Nval)  ]
y_test  = y1[(Nsamples-Ntest):Nsamples  ]

### Entrenamiento de la red neuronal usando validación bootstraping, normalización z-scope

In [7]:
all_sizes = []
all_sizes.append([7])
all_sizes.append([15])
all_sizes.append([31])
all_sizes.append([63])
all_sizes.append([127])
all_sizes.append([7,7])
all_sizes.append([7,15])
all_sizes.append([7,31])
all_sizes.append([7,63])
all_sizes.append([15,15])
all_sizes.append([15,31])
all_sizes.append([7,7,7])
w_ = []
error = []
for L in all_sizes:
    [w,e] = RedNeuronalTrain(X_train,y_train,Lsizes=L,outputs=12,alpha=0.1,epochs=50)
    print('para sizes = ',L,', da un error de ', e)
    w_.append(w)
    error.append(e)
    

para sizes =  [7] , da un error de  41.591155300645
para sizes =  [15] , da un error de  22.35250885712906
para sizes =  [31] , da un error de  15.758225012526864
para sizes =  [63] , da un error de  14.742345492381247
para sizes =  [127] , da un error de  24.799583068508525
para sizes =  [7, 7] , da un error de  254.4405409983704
para sizes =  [7, 15] , da un error de  157.81245378123046
para sizes =  [7, 31] , da un error de  3464.092128753793
para sizes =  [7, 63] , da un error de  8101.4999965565685
para sizes =  [15, 15] , da un error de  183.73433054737342
para sizes =  [15, 31] , da un error de  6082.348722339589
para sizes =  [7, 7, 7] , da un error de  328.3940446751657


In [8]:
predicted1      = []
Accuracy_train  = []
for j in range(0,len(all_sizes)):
    p1 = []
    for i in range(0,X_train.shape[0]):
        p1.append(np.argmax(Predict(X_train[i,:],w_[j])))
    predicted1.append(p1)
    Accuracy_train.append(np.sum(np.array(p1) == y_train)/X_train.shape[0])

In [9]:
predicted2      = []
Accuracy_val    = []
for j in range(0,len(all_sizes)):
    p2 = []
    for i in range(0,X_val.shape[0]):
        p2.append(np.argmax(Predict(X_val[i,:],w_[j])))
    predicted2.append(p2)
    Accuracy_val.append(np.sum(np.array(p2) == y_val)/X_val.shape[0])

In [10]:
d = {'1.topologia': all_sizes,'2. tasa de entrenamiento': Accuracy_train, '3. tasa de validación': Accuracy_val}
df = pd.DataFrame(data=d)
df

,1.topologia,2. tasa de entrenamiento,3. tasa de validación
0,[7],0.985064,0.933333
1,[15],0.991853,0.949206
2,[31],0.991853,0.939683
3,[63],0.994569,0.952381
4,[127],0.983707,0.936508
5,"[7, 7]",0.704684,0.707937
6,"[7, 15]",0.827563,0.825397
7,"[7, 31]",0.081466,0.079365
8,"[7, 63]",0.504413,0.536508
9,"[15, 15]",0.830278,0.815873


In [11]:
predicted_test = []
for i in range(0,X_test.shape[0]):
    predicted_test.append(np.argmax(Predict(X_test[i,:],w_[3])))
Accuracy_test = np.sum(np.array(predicted_test) == y_test)/X_test.shape[0]
print('Error de testing',100*(1.0-Accuracy_test),'%')
d = {'1. Label de prueba': y_test, '2. predicción final': predicted_test}
df = pd.DataFrame(data=d)
df

Error de testing 4.100946372239744 %


,1. Label de prueba,2. predicción final
0,0.0,0
1,6.0,6
2,0.0,0
3,0.0,0
4,0.0,0
5,6.0,6
6,0.0,0
7,0.0,0
8,0.0,0
9,0.0,0
